In [1]:
from operator import add, gt, sub
from unification.core import _unify, _reify, construction_sentinel

from kanren import eq, membero, run, unifiable, var, unify, reify
from kanren.core import lall
from kanren.term import applyo, term  # noqa: F401

import ibis
import ibis.expr.operations as ops
import ibis.expr.datatypes as dt
import ibis.expr.rules as rlz
from ibis.common.grounds import Pattern, Annotable
from collections.abc import Mapping

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
table = ibis.table([
    ("a", dt.int32), 
    ("b", dt.float64)
])
column = table['a'].name("a_").op()

In [3]:
ops.TableColumn[var(), var()]

TableColumn(table=~_1, name=~_2)

In [4]:
ops.TableColumn.pattern(var(), name=var())

TableColumn(table=~_3, name=~_4)

In [5]:
column

Alias(arg=TableColumn(table=UnboundTable(schema=ibis.Schema {
  a  int32
  b  float64
}, name='unbound_table_0'), name='a'), name='a_')

In [6]:
# @_unify.register(Annotable, Pattern, Mapping)
# def _unify_Annotable(u, v, s):
#     return _unify(u.args, v.args, s)

# @_unify.register(Pattern, Annotable, Mapping)
# def _unify_Pattern(u, v, s):
#     return _unify(u.args, v.args, s)

# @_reify.register(Pattern, Mapping)
# def _reify_Pattern(o, s):
#     kwargs = yield _reify(o._args, s) 
#     obj = o._cls(**kwargs)  
#     yield construction_sentinel
#     yield obj

In [6]:
t, n = var("t"), var("n")

unify(ops.TableColumn.pattern(t, name=n), column.arg)

{~t: UnboundTable(schema=ibis.Schema {
   a  int32
   b  float64
 }, name='unbound_table_0'),
 ~n: 'a'}

In [7]:
reify(ops.TableColumn.pattern(t, name=n), {t: column.arg.table, n: column.arg.name})

TableColumn(table=UnboundTable(schema=ibis.Schema {
  a  int32
  b  float64
}, name='unbound_table_0'), name='a')

In [8]:
x, y = var("x"), var("y")

run(
    0, 
    y, 
    eq(ops.Alias.pattern(x, name="a_"), column), 
    eq(ops.TableColumn.pattern(y, name=var()), x)
)[0]

UnboundTable(schema=ibis.Schema {
  a  int32
  b  float64
}, name='unbound_table_0')

In [9]:
run(
    0, 
    y, 
    eq(ops.Alias.pattern(ops.TableColumn.pattern(y, name=var()), name="a_"), column), 
)

(UnboundTable(schema=ibis.Schema {
   a  int32
   b  float64
 }, name='unbound_table_0'),)

In [10]:
class PhysicalColumn(ops.Node):
    table = rlz.instance_of(str)
    name = rlz.instance_of(str)

In [11]:
z = var("z")
n = var("n")

run(
    0, 
    z, 
    eq(ops.Alias.pattern(ops.TableColumn.pattern(y, name=var()), name=x), column), 
    eq(ops.UnboundTable.pattern(schema=var(), name=n), y),
    eq(PhysicalColumn.pattern(n, name=x), z),

)[0]

PhysicalColumn(table='unbound_table_0', name='a_')

In [12]:
# Alias(something, )

In [13]:
from ibis.expr.operations import Alias, UnboundTable, TableColumn

run(
    0, 
    z, 
    eq(Alias.match(TableColumn.match(y, name=var()), name=x), column), 
    eq(UnboundTable.match(schema=var(), name=n), y),
    eq(PhysicalColumn.match(n, name=x), z)
)[0]

PhysicalColumn(table='unbound_table_0', name='a_')

In [36]:
run(
    0, 
    z, 
    eq(ops.Alias[ops.TableColumn[ops.UnboundTable[var(), x], var()], y], column),
    eq(PhysicalColumn[x, y], z)
)[0]

PhysicalColumn(table='unbound_table_1', name='a_')

In [ ]:
Alias.match()